# This is the how you can pull data from the API

Make sure that you're staging/production credentials `yaml` is structured like so:

```
access_token_id: API_TOKEN_ID
access_token_secret: API_TOKEN_SECRET
```

In [245]:
import datetime as dt
import io
import json
import jsbeautifier
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
from runeq import Config, stream
from scipy import stats
import yaml
from IPython.display import display, Markdown
from IPython.core import display as ICD

In [251]:
def run_test_requests(
    params: dict, 
    headers, 
    endpoint, 
    field_name, 
    env='staging', 
    extra=True
):
    """
    Run a full suite of tests.
    params
        params for the request
    headers:
        client / user credentials
    endpoint:
        name of endpoint that you are querying from e.g. band_power, probability_symptom
    field_name
        name of the measurement e.g. power, probability
    env
        staging, production, or local
    extra
        runs tests for extra querying expressions e.g. first, last, mean, median
    """
    
    if env=='local':
        json_url = f'http://localhost:8080/v1/{endpoint}.csv'
        json_url = f'http://localhost:8080/v1/{endpoint}.json'

    elif env=='staging':
        json_url = f'https://stream-staging.runelabs.io/v1/{endpoint}.json'
        csv_url = f'https://stream-staging.runelabs.io/v1/{endpoint}.csv'

    elif env=='prod':
        json_url = f'https://stream.runelabs.io/v1/{endpoint}.json'
        csv_url = f'https://stream.runelabs.io/v1/{endpoint}.csv'
    else:
        raise ValueError('Unknown envinrontment')

    display(Markdown('#### JSON'))
    json_params = params
    availability_params = json_params | {'expression': f'availability({field_name})'}
    response = requests.get(json_url, params=json_params, headers=headers)
    availability_response = requests.get(json_url, params=availability_params, headers=headers)
    display(Markdown(f'##### {field_name}:'))
    ppjson(response.json())
    display(Markdown(f'##### availability({field_name}):'))
    ppjson(availability_response.json())

    if extra:
        for exp in ['first', 'mean', 'median', 'last']:
            exp_params = json_params | {'expression': f'{exp}({field_name})'}
            resp = requests.get(json_url, params=exp_params, headers=headers)
            display(Markdown(f'##### {exp}({field_name}):'))
            ppjson(resp.json())

    csv_params = {k:v for k,v in json_params.items() if k != 'partition_size'}
    availability_csv_params = csv_params | {'expression': f'availability({field_name})'}
    response = requests.get(csv_url, params=csv_params, headers=headers)
    availability_response = requests.get(
        csv_url, params=availability_csv_params, headers=headers)
    
    display(Markdown('#### CSV'))
    display(Markdown(f'##### {field_name}:'))
    ppcsv(response=response)
    display(Markdown(f'##### availability({field_name}):'))
    ppcsv(response=availability_response)


    if extra:
        for exp in ['first', 'mean', 'median', 'last']:
            exp_params = csv_params | {'expression': f'{exp}({field_name})'}
            resp = requests.get(csv_url, params=exp_params, headers=headers)
            display(Markdown(f'##### {exp}({field_name}):'))
            ppcsv(response=resp)

In [234]:
def ppjson(*args):
    """
    Pretty prints JSON so that long array are printed on one line
    """
    new_args = []
    for i, arg in enumerate(args):
        if type(arg) == dict:
            arg = jsbeautifier.beautify(json.dumps(arg), options)
        new_args.append(arg)
    print(*new_args)
    print('\n')

In [241]:
def ppcsv(*args, response):
    """
    Prints CSV in a dataframe format
    """
    print(*args)
    df = pd.read_csv(io.StringIO(response.text))
    display(df)
    print('\n')

## Local

#### v1/event.json

In [94]:
params = {
    'patient_id': 'dangerzonepatient', 
    'start_time': '1567566520',
    'end_time': '1567566530',
    'device_id': 'kp0ArnOK', #fake_percept
    'event': 'clinical'
}
response = requests.get('http://localhost:8080/v1/event.json', params=params)

## Staging

In [8]:
config_path = '/Users/poorv/.rune/api-config-staging.yaml'
with open(config_path) as f:
    credentials = yaml.safe_load(f)
    
headers = {
    'X-Rune-User-Access-Token-Id': credentials['access_token_id'],
    'X-Rune-User-Access-Token-Secret': credentials['access_token_secret']
}

#### v1/accel

In [252]:
params = {
    'device_id': 'OXWME382',
    'patient_id': 'acd84ff341f54588a0eb6afb1173a2f1',
    'start_time': '1626128333',
    'end_time': '1626147851',
   'resolution': '6000',
}
run_test_requests(params, headers, 'accel', 'accel')

#### JSON

##### accel:

{
  "result": {
    "cardinality": 4,
    "time": [1626126000, 1626132000, 1626138000, 1626144000],
    "x": [-0.6352692246437073, 0.11776725947856903, -0.04504386708140373, -0.5210418701171875],
    "y": [0.2354431450366974, 1.0323182344436646, 0.4831390678882599, 0.6363219618797302],
    "z": [-0.7210692763328552, 0.12341313064098358, -0.8650969862937927, -0.1139983981847763]
  },
  "success": true
}




##### availability(accel):

{
  "result": {
    "availability(accel)": [1, 1, 1, 1],
    "cardinality": 4,
    "time": [1626126000, 1626132000, 1626138000, 1626144000]
  },
  "success": true
}




...

,time,x,y,z
0,1.626126e+09,0.104360,0.565025,-0.005304
1,1.626132e+09,0.362081,0.579228,-0.111047
2,1.626138e+09,0.119065,0.553347,-0.254878
3,1.626144e+09,-0.002121,0.579046,-0.106290


##### median(accel):

,time,x,y,z
0,1.626126e+09,-0.027790,0.519338,-0.044327
1,1.626132e+09,0.345081,0.589878,-0.143318
2,1.626138e+09,0.035135,0.694097,-0.313926
3,1.626144e+09,-0.033143,0.707728,-0.173384


##### last(accel):

,time,x,y,z
0,1.626126e+09,0.054230,0.950684,0.060516
1,1.626132e+09,-0.062164,0.481400,-0.854019
2,1.626138e+09,-0.517670,0.587143,-0.116455
3,1.626144e+09,0.055313,-0.226288,-0.977539


#### v1/band_power

In [ ]:
params = {
    'patient_id': '1ce0cbb08eeb4986af2ee7d965e0f505',
    'start_time': '1583357120',
    'end_time': '1583366377',
    'device_id': 'kp0ArnOK'
}
run_test_requests(params, headers, 'band_power', 'power', extra=False)

#### v1/probability_symptom

In [ ]:
params = {
    'device_id': 'kp0ArnOK',
    'symptom': 'tremor',
    'patient_id': '1ce0cbb08eeb4986af2ee7d965e0f505',
    'start_time': '1586210745',
    'end_time': '1586229747',
    'resolution': '3000',
    'partition_size': '288',
    'timestamp': 'datetime',
    'timezone': '-14400',
    'severity': '*'
}
run_test_requests(params, headers, 'probability_symptom', 'probability')

## Production

In [236]:
prod_config_path = '/Users/poorv/.rune/api-config-production.yaml'
with open(prod_config_path) as f:
    production_credentials = yaml.safe_load(f)
    
prod_headers = {
    'X-Rune-User-Access-Token-Id': production_credentials['access_token_id'],
    'X-Rune-User-Access-Token-Secret': production_credentials['access_token_secret']
}

#### v1/band_power

In [ ]:
prod_params = {
    'patient_id': '9c2ffb54dcab4d54a3dbd05770c2c1c5',
    'start_time': '1583357120',
    'end_time': '1583366377',
    'device_id': '0uuQAj6X'
}
run_test_requests(prod_params, prod_headers, 'band_power', 'power', extra=False, env="prod")

In [ ]:
prod_params = {
    'device_id': '0uuQAj6X',
    'symptom': 'tremor',
    'patient_id': '9c2ffb54dcab4d54a3dbd05770c2c1c5',
    'start_time': '1586210745',
    'end_time': '1586229747',
    'resolution': '3000',
    'partition_size': '288',
    'timestamp': 'datetime',
    'timezone': '-14400',
    'severity': '*'
}
run_test_requests(prod_params, prod_headers, 'probability_symptom', 'probability', env='prod')